# Introduction

After first attempts, the score was very low. (about 0.5)

Then, we lokked up other Kaggler's submissions, in particular Beatriz Justino's https://www.kaggle.com/code/beatrizjustino/lstm-disaster-tweets. (Version 12)

And we wondered, why it was so much better (about 0.8). Here are the findngs:

# Imports

In [11]:
from sklearn.model_selection import train_test_split
from tensorflow import keras
import tensorflow as tf
import numpy as np
import pandas as pd

# Imports that Justino's code needs:
from keras import preprocessing
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Embedding, LSTM, Dense, Dropout,BatchNormalization
from keras.models import Sequential
import tensorflow as tf
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

# Choices

In [28]:
BATCH_SIZE = 32
MAX_TOKENS = 10000
EPOCHS = 20
MAX_SENTENCE_LENGTH = 60

# Raw Datasets

In [13]:
all_known_df = pd.read_csv("/kaggle/input/nlp-getting-started/train.csv")
test_df = pd.read_csv("/kaggle/input/nlp-getting-started/test.csv")

train_df, val_df = train_test_split(all_known_df, test_size=0.2, random_state=42)

## Note that the data seems to be badly classified:
Two examples classified as disaster, which I stumbled about:

In [14]:
badly_classified_example = all_known_df[all_known_df["text"].str.contains("whirlwind of time")]
print(badly_classified_example)
print(badly_classified_example['text'].to_numpy())
print(badly_classified_example['target'].to_numpy())

badly_classified_example = all_known_df[all_known_df["text"].str.contains("Attack II Volleyball")]
print()
print(print(badly_classified_example))
print(badly_classified_example['text'].to_numpy())
print(badly_classified_example['target'].to_numpy())

         id    keyword                        location  \
7270  10409  whirlwind  Stamford & Cork (& Shropshire)   

                                                   text  target  
7270  I moved to England five years ago today. What ...       1  
['I moved to England five years ago today. What a whirlwind of time it has been! http://t.co/eaSlGeA1B7']
[1]

      id keyword location                                               text  \
466  674  attack      NaN  #volleyball Attack II Volleyball Training Mach...   

     target  
466       1  
None
['#volleyball Attack II Volleyball Training Machine - Sets Simulation - http://t.co/dCDeCFv934 http://t.co/dWBC1dUvdk']
[1]


# Justino's Raw Dataset

Justino encodes the target differently, and balances the uneven classes. (It does not seem to help a lot.)

In [15]:
def get_Justino_raw_data(df, df_val=None):
    def extract_text_and_target(df):
        return df["text"].to_numpy(), df["target"].to_numpy()
    class_0 = df[df['target'] == 0]
    class_1 = df[df['target'] == 1]
    class_0_under = class_0.sample(len(class_1), random_state=42)
    df_equalized = pd.concat([class_0_under, class_1])
    text, label = extract_text_and_target(df_equalized)
    if df_val is not None:
        text_val, label_val = extract_text_and_target(df_val)
        return text, label, text_val, label_val
    else:
        return text, label

# Our preprocessing functions

One function for cleaning text snippets:

In [19]:
import re, string
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
stemmer = SnowballStemmer('english')
stop_words = set(stopwords.words('english'))

def clean_stem_text(text):    
    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    # Remove @mentions and hashtags
    text = re.sub(r'\@\w+|\#', '', text)
    # Remove digits
    text = re.sub(r'\d+', '', text)
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()
    # Convert to lowercase
    text = text.lower()
    # Remove stopwords and stem
    text = ' '.join([stemmer.stem(word) 
                     for word in text.split() 
                     if word not in stop_words])
    return text

def clean_stem_byte_tensor(tensor):
    texts = tensor.numpy()
    texts = np.reshape(texts, (-1,))
    texts = [text.decode("utf-8") for text in texts]
    texts = [clean_stem_text(text) for text in texts]
    return tf.constant(texts, dtype=tf.string, shape=(len(texts),))

Lots of functions for applying the cleaning function and for encoding the text numerically:

(One can later chose from these different methods.)

In [20]:
int_text_vectorization_custom_clean_text = keras.layers.TextVectorization(
    output_mode="int",
    max_tokens=MAX_TOKENS,
    output_sequence_length=MAX_SENTENCE_LENGTH,
    standardize=clean_stem_byte_tensor
)
tf_idf_text_vectorization_custom_clean_text = keras.layers.TextVectorization(
    output_mode="tf_idf",
    max_tokens=MAX_TOKENS,
    standardize=clean_stem_byte_tensor
)
int_text_vectorization = keras.layers.TextVectorization(
    output_mode="int",
    max_tokens=MAX_TOKENS,
    output_sequence_length=MAX_SENTENCE_LENGTH
)
tf_idf_text_vectorization = keras.layers.TextVectorization(
    output_mode="tf_idf",
    max_tokens=MAX_TOKENS
)
vectorizers = [int_text_vectorization_custom_clean_text,
              tf_idf_text_vectorization_custom_clean_text,
              int_text_vectorization,
              tf_idf_text_vectorization]

def preprocess_df(df, df_val=None, mode="int", clean="standard", df_test=None):
    if mode=="int" and clean=="standard":
        vectorizer = int_text_vectorization
    elif mode=="tf_idf" and clean=="standard":
        vectorizer = tf_idf_text_vectorization
    elif mode=="int" and clean=="custom":
        vectorizer = int_text_vectorization_custom_clean_text
    elif mode=="tf_idf" and clean=="custom":
        vectorizer = tf_idf_text_vectorization_custom_clean_text
    else:
        raise ValueError("Value not allowed")
        
    def df_to_X_y(df):
        X = df["text"].to_numpy()
        y = np.reshape(np.array(df["target"]), (-1,1))
        return X, y
    
    X, y = df_to_X_y(df)
    vectorizer.adapt(X)
    if df_test is not None:
        X_test = vectorizer(df_test["text"].to_numpy())
        if df_val is not None:
            X_val, y_val = df_to_X_y(df_val)
            return vectorizer(X), y, vectorizer(X_val), y_val, X_test
        else:
            return vectorizer(X), y, X_test
    else:
        if X_val is not None:
            X_val, y_val = df_to_X_y(df_val)
            return vectorizer(X), y, vectorizer(X_val), y_val
        else:
            return vectorizer(X), y

(The proprocessing is safely run on the dataset (not in the model) to save the GPU from being blocked.)

# Justino's preprocessing functions
ATTENTION: There seemed to be a bug in applying the stemming! It does not work with lists, but should be applied on single words.

In [21]:
def clean_Justino(text_array):
    def process_sentence(sentence):
        sentence = re.sub(r'http\S+|www\S+|https\S+', '', sentence, flags=re.MULTILINE)
        sentence = re.sub(r'#\S+', '', sentence)
        sentence = sentence.translate(str.maketrans('', '', string.punctuation))
        words = sentence.split()
        filtered_words = [word for word in words if word.lower() not in ENGLISH_STOP_WORDS]
        return ' '.join(filtered_words)
    return [process_sentence(sentence) for sentence in text_array]

def get_Justino_preprocessed_data(df, df_val=None, label_to_cat=True, vocab_length=None, length_long_sentence=MAX_SENTENCE_LENGTH):
    if df_val is not None:
        texts, labels, texts_val, labels_val = get_Justino_raw_data(df, df_val)
        texts_val = clean_Justino(texts_val)
        texts_val = [stemmer.stem(word) for word in texts_val]
    else:
        texts, labels = get_Justino_raw_data(df)
    texts = clean_Justino(texts)
    #print(texts[:5])
    texts = [stemmer.stem(word) for word in texts]
    #print("-----------")
    #print(texts[:5])
    
    word_tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=vocab_length)
    word_tokenizer.fit_on_texts(texts)
    
    #from nltk.tokenize import word_tokenize
    #longest_train = max(text, key=lambda sentence: len(word_tokenize(sentence)))
    #length_long_sentence = len(word_tokenize(longest_train))
    
    def process_data(texts, labels):
        sequences = word_tokenizer.texts_to_sequences(texts)
        padded_sequences = pad_sequences(sequences, maxlen=length_long_sentence)
        if label_to_cat:
            labels = keras.utils.to_categorical(labels)
        return padded_sequences, labels
    
    padded_sequences, labels = process_data(texts, labels)
    if df_val is not None:
        padded_sequences_val, labels_val = process_data(texts_val, labels_val)
        return padded_sequences, labels, padded_sequences_val, labels_val
    else:
        return padded_sequences, labels
#get_Justino_preprocessed_data(train_df, val_df)

# F1-Metric

Using metrics=["f1_score"] is not working properly in tensorflow up until now!! The problem is discussed on some GitHub-site...

We specify an f1-metric more manually here:

In [22]:
f1 = tf.keras.metrics.F1Score(
    average=None, threshold=0.5, name='f1', dtype=None
)

For training, one could also try to use an adapted "f1-loss":

In [23]:
def f1_loss(y_true, y_pred):
    
    tp = tf.sum(tf.cast(y_true*y_pred, 'float'), axis=0)
    tn = tf.sum(tf.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = tf.sum(tf.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = tf.sum(tf.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + tf.epsilon())
    r = tp / (tp + fn + tf.epsilon())

    f1 = 2*p*r / (p+r+tf.epsilon())
    f1 = tf.where(tf.math.is_nan(f1), tf.zeros_like(f1), f1)
    return 1 - tf.mean(f1)


# Our Models

In [24]:

def get_model(max_tokens, hidden_dim=16, embedding_dim=33):
    inputs = keras.Input(shape=(MAX_SENTENCE_LENGTH,), dtype="int64")
    x = keras.layers.Embedding(input_dim=max_tokens, output_dim=embedding_dim)(inputs)
    #x = keras.layers.Flatten()(x)
    #x = keras.layers.Conv1D(1,embedding_dim)(x)[:,0]
    x = keras.layers.GlobalAveragePooling1D()(x)
    x = keras.layers.Dense(hidden_dim, activation="relu")(x)
    x = keras.layers.Dense(hidden_dim, activation="relu")(x)+x
    #x = keras.layers.Dense(hidden_dim, activation="relu")(x)
    x = keras.layers.Dropout(0.5)(x)
    x = keras.layers.Dense(1)(x)
    model = keras.Model(inputs=inputs, outputs=x)
    model.compile(loss=keras.losses.BinaryCrossentropy(from_logits=True),
                 #loss=f1_loss,
                 optimizer="rmsprop",
                 metrics=["accuracy", f1],
                 #run_eagerly=True
                 )
    return model

# We should also try to do simple logistic regression, when we find the time for it:

#def get_LR_model(max_tokens):
#    inputs = keras.Input(shape=(MAX_SENTENCE_LENGTH,), dtype="int64")
#    x = tf.keras.layers.CategoryEncoding(
#        num_tokens=max_tokens, output_mode='multi_hot', sparse=False
#    )(inputs)
#    x = keras.layers.Dense(1)(x)
#    model = keras.Model(inputs=inputs, outputs=x)
#    model.compile(loss=keras.losses.BinaryCrossentropy(from_logits=True),
#                 #loss=f1_loss,
#                 optimizer="adam",
#                 metrics=["f1_score", "accuracy", "precision", "recall"],
#                 #run_eagerly=True
#                 )
#    return model

#def get_tf_idf_LR_model(max_tokens):
#    inputs = keras.Input(shape=(max_tokens,), dtype="int64")
#    x = keras.layers.Dense(1)(inputs)
#    model = keras.Model(inputs=inputs, outputs=x)
#    model.compile(loss=keras.losses.BinaryCrossentropy(from_logits=True),
#                 #loss=f1_loss,
#                 optimizer="adam",
#                 metrics=["f1_score", "accuracy", "precision", "recall"],
#                 #run_eagerly=True
#                 )
#    return model

In [37]:
# That's basically Justino's model for our own data-formatting.. Maybe we'll use it one day.

#def get_LSTM_model():
#    inputs = keras.Input((MAX_SENTENCE_LENGTH,), dtype=tf.int64)
#    x = keras.layers.Embedding(input_dim=MAX_TOKENS, output_dim=128)(inputs)
#    x = keras.layers.Dropout(0.5)(x)
#    x = keras.layers.LSTM(65)(x)
#    x = keras.layers.Dense(1, activation="sigmoid")(x)
#    model=keras.Model(inputs, x)
#    model.compile(loss=keras.losses.BinaryCrossentropy(from_logits=False),
#                  optimizer="Adamax",
#                  metrics=["accuracy", "f1_score"],
#                 #run_eagerly=True
#                 )
#    model.summary()
#    return model



# Jutino's Model

In [26]:
def get_Justino_model(max_words=MAX_TOKENS, max_len=MAX_SENTENCE_LENGTH):
    model = Sequential()
    model.add(Embedding(max_words, 128))
    model.add(Dropout(0.5))
    model.add(LSTM(64))
    model.add(Dense(2, activation='sigmoid'))

    model.compile(loss='BinaryCrossentropy', optimizer='Adamax', metrics=['accuracy', f1])
    return model

## Runs

Justino orginal:

In [29]:
X_Justino, y_Justino = get_Justino_preprocessed_data(all_known_df)
mod_Justino = get_Justino_model(max_words=np.max(np.max(X_Justino))+1)
mod_Justino.fit(X_Justino, y_Justino, epochs=EPOCHS, batch_size=64)

Epoch 1/20
103/103 ━━━━━━━━━━━━━━━━━━━━ 9s 64ms/step - accuracy: 0.5415 - f1: 0.5381 - loss: 0.6898
Epoch 2/20
103/103 ━━━━━━━━━━━━━━━━━━━━ 10s 62ms/step - accuracy: 0.6776 - f1: 0.6748 - loss: 0.6206
Epoch 3/20
103/103 ━━━━━━━━━━━━━━━━━━━━ 6s 57ms/step - accuracy: 0.7737 - f1: 0.7752 - loss: 0.4896
Epoch 4/20
103/103 ━━━━━━━━━━━━━━━━━━━━ 7s 66ms/step - accuracy: 0.8080 - f1: 0.8070 - loss: 0.4240
Epoch 5/20
103/103 ━━━━━━━━━━━━━━━━━━━━ 6s 62ms/step - accuracy: 0.8390 - f1: 0.8386 - loss: 0.3719
Epoch 6/20
103/103 ━━━━━━━━━━━━━━━━━━━━ 10s 57ms/step - accuracy: 0.8577 - f1: 0.8584 - loss: 0.3334
Epoch 7/20
103/103 ━━━━━━━━━━━━━━━━━━━━ 6s 54ms/step - accuracy: 0.8766 - f1: 0.8769 - loss: 0.2972
Epoch 8/20
103/103 ━━━━━━━━━━━━━━━━━━━━ 6s 55ms/step - accuracy: 0.8912 - f1: 0.8917 - loss: 0.2769
Epoch 9/20
103/103 ━━━━━━━━━━━━━━━━━━━━ 6s 59ms/step - accuracy: 0.8954 - f1: 0.8953 - loss: 0.2551
Epoch 10/20
103/103 ━━━━━━━━━━━━━━━━━━━━ 6s 57ms/step - accuracy: 0.9095 - f1: 0.9094 - loss: 0.22

Justino with splitting data into train and val:

In [30]:
X_Justino, y_Justino, X_Justino_val, y_Justino_val = get_Justino_preprocessed_data(train_df, val_df)
mod_Justino = get_Justino_model(max_words=np.max(np.max(X_Justino))+1)
mod_Justino.fit(X_Justino, y_Justino, validation_data=(X_Justino_val, y_Justino_val), epochs=EPOCHS, batch_size=64)

Epoch 1/20
82/82 ━━━━━━━━━━━━━━━━━━━━ 8s 64ms/step - accuracy: 0.5634 - f1: 0.8516 - loss: 0.6891 - val_accuracy: 0.6842 - val_f1: 0.6604 - val_loss: 0.6523
Epoch 2/20
82/82 ━━━━━━━━━━━━━━━━━━━━ 10s 61ms/step - accuracy: 0.6683 - f1: 0.6649 - loss: 0.6392 - val_accuracy: 0.7413 - val_f1: 0.7179 - val_loss: 0.5537
Epoch 3/20
82/82 ━━━━━━━━━━━━━━━━━━━━ 5s 62ms/step - accuracy: 0.7497 - f1: 0.7487 - loss: 0.5168 - val_accuracy: 0.7735 - val_f1: 0.7515 - val_loss: 0.4887
Epoch 4/20
82/82 ━━━━━━━━━━━━━━━━━━━━ 5s 59ms/step - accuracy: 0.8094 - f1: 0.8076 - loss: 0.4222 - val_accuracy: 0.7761 - val_f1: 0.7673 - val_loss: 0.4755
Epoch 5/20
82/82 ━━━━━━━━━━━━━━━━━━━━ 5s 59ms/step - accuracy: 0.8560 - f1: 0.8548 - loss: 0.3462 - val_accuracy: 0.7794 - val_f1: 0.7726 - val_loss: 0.4845
Epoch 6/20
82/82 ━━━━━━━━━━━━━━━━━━━━ 5s 59ms/step - accuracy: 0.8703 - f1: 0.8699 - loss: 0.3239 - val_accuracy: 0.7814 - val_f1: 0.7719 - val_loss: 0.4900
Epoch 7/20
82/82 ━━━━━━━━━━━━━━━━━━━━ 5s 58ms/step - accu

### Own models:

In [32]:
X_train, y_train, X_val, y_val, X_test = preprocess_df(train_df, val_df, df_test=test_df)
model = get_model(MAX_TOKENS)
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=20, batch_size=64,
         callbacks=[keras.callbacks.ModelCheckpoint("own_model.keras",save_best_only=True)])

Epoch 1/20
96/96 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.5759 - f1: 0.2025 - loss: 0.6889 - val_accuracy: 0.5739 - val_f1: 0.0000e+00 - val_loss: 0.6813
Epoch 2/20
96/96 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.5725 - f1: 0.0000e+00 - loss: 0.6850 - val_accuracy: 0.5739 - val_f1: 0.0000e+00 - val_loss: 0.6824
Epoch 3/20
96/96 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.5587 - f1: 0.0000e+00 - loss: 0.6862 - val_accuracy: 0.5739 - val_f1: 0.0000e+00 - val_loss: 0.6802
Epoch 4/20
96/96 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.5714 - f1: 0.0000e+00 - loss: 0.6819 - val_accuracy: 0.5739 - val_f1: 0.0000e+00 - val_loss: 0.6838
Epoch 5/20
96/96 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.5649 - f1: 0.0000e+00 - loss: 0.6841 - val_accuracy: 0.5739 - val_f1: 0.0000e+00 - val_loss: 0.6788
Epoch 6/20
96/96 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.5649 - f1: 0.0000e+00 - loss: 0.6803 - val_accuracy: 0.5739 - val_f1: 0.0000e+00 - val_loss: 0.6756
Epoch 7/20
96/96 ━━

# Helper function for recunstructing Reconstruct Sentences

In [ ]:
def reconstruct_sentence(int_vec, vectorizer):
    voc = np.array(vectorizer.get_vocabulary())
    #print(int_vecs)
    #print(type(int_vecs))
    sentence = (" ".join(voc[int_vec]))
    return sentence
#for i in range(...):
#    print(reconstruct_sentence(...), "  ", y_train[i])

# Submit

In [ ]:
model = keras.models.load_model("own_model.keras")
predictions_logits = model.predict(X_test)
predictions = tf.math.sigmoid(predictions_logits).numpy()
submit_df = test_df[["id"]].copy()
submit_df["target"] = (predictions>0.5).astype(int)
submit_df.to_csv("submission.csv", index=False)
submit_df.head()

# Conclusion

It seems, that our own original submission where heavily overfitted.

Using the ModelCheckpoint-Callback helps a lot! (What else do we have the validation data for?!)

Also, Justino's training starts to improve MUCH earlier, probably because of the ordering in the data, leading to some kind of "Curriculum learning". 
(See the code line with
df_equalized = pd.concat([class_0_under, class_1]))